# ✋ **Z punktów do gestów – rozpoznawanie dłoni w akcji!**
Cześć! Miło nam powitać Was na naszym workshopie dotyczącym wizji komputerowej, a konkretnie - klasyfikacji gestów. W tym notatniku postaramy się zaimplementować rozwiązanie, które pozwoli w czasie rzeczywistym odgadywać gest pokazywany przez Ciebie do kamery. Jeśli cokolwiek będzie niejasne, nie wahaj się pytać :)

## **0. Zapoznanie z MediaPipe**
Czas wykryć pierwsze punkty kluczowe! Uruchom plik `data_collection.py`, a rozpocznie się zczytywanie obrazu z Twojej kamery. Możesz włączyć lub wyłączyć dodatkowe wyświetlanie punktów wciskając 'k', a wyjść przy użyciu 'q'.

## **1. Dane**
Aby rozwiązać nasz problem wykorzystując nadzorowane uczenie maszynowe, naturalnie będziemy potrzebowali zbioru oetykietowanych danych. W tym celu zebraliśmy kilkadziesiąt klatek, na których pokazujemy 5 różnych gestów:
- **palm up** ✋
- **fist** 👊
- **peace** ✌
- **thumbs up** 👍
- **ok sign** 👌<br/>

Aby uzyskać do nich dostęp, wczytaj i wyświetl zawartość pliku `data.csv`:

In [1]:
import pandas as pd

url = "data.csv"
df = pd.read_csv(url)

X = df.drop("label", axis=1).drop("timestamp", axis=1).values
y = df["label"].values
df.head(5)

,timestamp,label,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,...,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,20250527_114502_827409,0,0.372025,0.719961,4.393711e-07,0.300324,0.710500,-0.026703,0.247515,0.641015,...,-0.037881,0.373383,0.441559,-0.053515,0.369498,0.392890,-0.059776,0.365106,0.347981,-0.064498
1,20250527_114503_332402,0,0.413500,0.729521,3.262455e-07,0.340419,0.710898,-0.018543,0.293954,0.636054,...,-0.033006,0.418824,0.457132,-0.044465,0.413216,0.409872,-0.049255,0.406945,0.366703,-0.053574
2,20250527_114503_752013,0,0.449705,0.719101,3.897718e-07,0.381664,0.694543,-0.022093,0.338376,0.622084,...,-0.038472,0.467837,0.453915,-0.054453,0.464854,0.404276,-0.061642,0.460901,0.357389,-0.066819
3,20250527_114504_259373,0,0.500856,0.706181,2.969335e-07,0.435813,0.669880,-0.013869,0.404370,0.583814,...,-0.029998,0.540222,0.435370,-0.043824,0.540488,0.386746,-0.050771,0.539009,0.340207,-0.056621
4,20250527_114504_757549,0,0.549247,0.701973,2.553577e-07,0.490214,0.665783,-0.006732,0.465360,0.582481,...,-0.033257,0.568108,0.442185,-0.045474,0.566589,0.393813,-0.053515,0.562536,0.348694,-0.060820


In [ ]:
## **2. Model**
